<a href="https://colab.research.google.com/github/yinon2592/DL_Project_046211/blob/main/section_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
# drive.mount('/content/drive/my-drive/project_calculations')
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/investigating-sentiment-analysis/data/training.1600000.processed.noemoticon.csv.zip -P data
!unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip
! pip install transformers

File ‘data/training.1600000.processed.noemoticon.csv.zip’ already there; not retrieving.

Archive:  data/training.1600000.processed.noemoticon.csv.zip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

In [ ]:
### Prepare data
# lyrics = pd.read_csv('lyrics-data.csv', nrows=9000)
# #Only keep popular artists, with genre Rock/Pop and popularity high enough
# artists = pd.read_csv('artists-data.csv', nrows=9000)
# artists = artists[(artists['Genre'].isin(['Rock'])) & (artists['Popularity']>5)]
# df = lyrics.merge(artists[['Artist', 'Genre', 'Link']], left_on='ALink', right_on='Link', how='inner')
# df = df.drop(columns=['ALink','SLink','Idiom','Link'])

# #Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
# df = df[df['Lyric'].apply(lambda x: len(x.split(' ')) < 350)]

# print(df.head(5))

# #Create a very small test set to compare generated text with the reality
# test_set = df.sample(n = 200)
# df = df.loc[~df.index.isin(test_set.index)]

# #Reset the indexes
# test_set = test_set.reset_index()
# df = df.reset_index()

# #For the test set only, keep last 20 words in a new column, then remove them from original column
# test_set['True_end_lyrics'] = test_set['Lyric'].str.split().str[-20:].apply(' '.join)
# test_set['Lyric'] = test_set['Lyric'].str.split().str[:-20].apply(' '.join)



import os
import pandas as pd
from torch.utils.data import Dataset
import re
import torch
# Step 1: Dataset Preparation
# Step 2: Data Preprocessing
def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove usernames
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9]+', ' ', text)  # Remove special characters
    # Remove newlines and multiple whitespaces
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)

    text = text.lower().strip()
    return text
file_path="data/training.1600000.processed.noemoticon.csv"
df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
df = df[[0, 5]]
df.columns = ['label', 'text']
df = df.sample(100, random_state=1)

# # df['label'] = df['label'].replace({0: 'negative', 2: 'neutral', 4: 'positive'})
# df['label'] = df['label'].replace({0: 'negative', 2: 'neutral', 4: 1})
# # Drop the rows with 'neutral' sentiment
# df = df[df['label'] != 'neutral']
df = df[df['label'] != 2]
df['label'] = df['label'].replace({4: 1})
df['text'] = df['text'].apply(clean_text)
print(df.label.value_counts())
print(df.sample(16))

0    57
1    43
Name: label, dtype: int64
         label                                               text
1299511      1  wow thunder too my grandpa used to tell me the...
771870       0                                           so sorry
100900       0       awww we have same promblem my dear i hate it
1413143      1                                  all in for repair
1116729      1     getting ready for mtv movie awards see u there
1533289      1  checking to see who may have saude creek wine ...
931680       1  the bus was extremely early today bet few peop...
565077       0  finished homework 5 minutes after they made th...
832114       1  haha i know what a ridicukous scene love it th...
716696       0  i am watching churchonline but i can t get the...
668255       0  hmmm wonder if my plans will actually happen t...
639291       0  my fingers are like prunes waaaaa i hate that ...
1202697      1          just so you know i don t like you one bit
654427       0  think someone has 

In [ ]:
class SongLyrics(Dataset):
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['text']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)

    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]

dataset = SongLyrics(df['text'], truncate=True, gpt2_type="gpt2")

In [ ]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

model_path = '/content/drive/My Drive/project_calculations/generative_model.pth'
if os.path.exists(model_path):
  print("loading last model parameters..")
  model.load_state_dict(torch.load(model_path))

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=2, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100

    device = 'cuda' if torch.cuda.is_available() else 'cpu'


    # device=torch.device("cuda")
    # model = model.cuda()
    model.to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            # torch.save(
            #     model.state_dict(),
            #     os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            # )
            #     if val_acc > best_val:
            # best_val = val_acc
            # # Save your model to Google Drive
            # print(" current best val is ",best_val)
            torch.save(model.state_dict(), '/content/drive/My Drive/project_calculations/generative_model.pth')
    return model

In [ ]:
model = train(dataset=dataset, model=model, tokenizer=tokenizer, save_model_on_epoch=True, epochs=5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


100it [00:05, 19.82it/s]


Training epoch 1
tensor(3.0914, device='cuda:0', grad_fn=<NllLossBackward0>)


100it [00:01, 51.28it/s]
